<a href="https://colab.research.google.com/github/drcochran-newman/Data-Preprocessing/blob/main/Superstore_Log_Transforms_and_Binning_BEGIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Superstore Log Transforms and Binning

**May 2025**

by David Cochran

[Data Science @ Newman University](https://newmanu.edu/academics/graduate-programs/ms-data-science)


# Import Libraries and Set Notebook Defaults

In [ ]:
# Numpy and Pandas
import numpy as np
import pandas as pd

# Matplotlib and Seaborn
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# Jupyter Notebook customizations

# Remove scientific notation from descriptive stats
pd.options.display.float_format = '{:,.3f}'.format

# Display all columns of a dataframe
pd.set_option('display.max_columns', None)

# Widen columns
pd.set_option('display.max_colwidth', 150)

In [ ]:
# Customize Seaborn Plot Styles

# Adjust to retina quality
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats("retina")

# Adjust dpi and font size to support high-pixel-density screens
sns.set(rc={"figure.dpi":100, 'savefig.dpi':300})
sns.set_context('notebook', font_scale = 0.8)

# Display tick marks
sns.set_style('ticks')

# Remove borders
plt.rc('axes.spines', top=False, right=False, left=False, bottom=False)

# Set color palettes for plots
# See Matplotlib named color options here: https://matplotlib.org/stable/gallery/color/named_colors.html
blue = 'deepskyblue' # Use 'skyblue' for a lighter blue
orange = 'orange'
cp1 = [blue, orange]

# cp2 Palette - Reversed binary color order when needed for certain plots
cp2 = [orange, blue]

# cp5 Palette - 5 colors for use with categorical data
turquoise = 'mediumaquamarine'
salmon = 'darksalmon'
tan = 'tan'
gray = 'darkgray'
cp5 = [blue, turquoise, salmon, tan, gray]

# cpd Palette - blue-to-orange diverging palette for correlation heatmaps
cpd = sns.diverging_palette(242, 39, s=100, l=65, n=11)

# Set the default palette
sns.set_palette(cp1)

In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import Cleaned Data

In [ ]:
# Read from CSV
# Set appropriate data types

# Set file path
path = '/content/drive/MyDrive/Data Projects/Preprocessing/Superstore Preprocessing/data/Global_Superstore_Orders_2020_23.csv'

# Read data
# Set Postal Code datatype to string and fill to 5 digits with leading zeros
superstore_cleaned_df = pd.read_csv(path, dtype={'Postal Code': str})

# Fill postal codes to 5 characters with leading zeros when needed
superstore_cleaned_df['Postal Code'] = superstore_cleaned_df['Postal Code'].str.zfill(5)

# Convert dates to datetime
superstore_cleaned_df['Order Date'] = pd.to_datetime(superstore_cleaned_df['Order Date'])
superstore_cleaned_df['Ship Date'] = pd.to_datetime(superstore_cleaned_df['Ship Date'])

superstore_cleaned_df.head()

In [ ]:
superstore_cleaned_df.tail()

In [ ]:
# Get number of rows and columns
superstore_cleaned_df.shape

In [ ]:
# Dataframe fundamental info
superstore_cleaned_df.info()

In [ ]:
superstore_cleaned_df.describe()

In [ ]:
superstore_cleaned_df.describe(include='object')

# Create Labels Dataframe

We will create the first of two dataframes from superstore_cleaned_df:
- Customer_Labels

In [ ]:
# Create Labels Dataframe
# Include all records
# Three columns: CustomerID (index), Order Date Min, and Order Date Max

# Group by Customer and aggregate the Min and Max Order Date
Customer_Labels = superstore_cleaned_df.groupby('Customer ID').agg(
    Min_Order_Date=('Order Date', 'min'),
    Max_Order_Date=('Order Date', 'max')
)

# Preview results
Customer_Labels.sample(12)

**Eliminate all Customers who did not make a purchase before 2023.**

- We are only interested in customers who made a first purchase prior to 2023.

- Non-churners purchased before 2023 and then made another in 2023.

- Churners also purchased before 2023 but then did not return in 2023.

So: Eliminate all customers from our list who made no purchases before 2023.

In [ ]:
# Remove all customers whose Min Order Date year is 2023.

Customer_Labels = Customer_Labels[Customer_Labels['Min_Order_Date'].dt.year < 2023]

Customer_Labels.sample(12)


In [ ]:
# View Labels Dataframe Stats
Customer_Labels.describe()

In [ ]:
# Create Churn Target Variable
# If the Year of Max Order Date == 2023
# Then Churn = 0
# Else Churn = 1

Customer_Labels['Churn'] = np.where(Customer_Labels['Max_Order_Date'].dt.year == 2023, 0, 1)

Customer_Labels.head()

In [ ]:
# Drop the order date fields, so that we have only Customer ID and Churn
Customer_Labels = Customer_Labels.drop(columns=['Min_Order_Date', 'Max_Order_Date'])
Customer_Labels.head(12)

In [ ]:
# Compare number of Churn = 1 versus Churn = 0
Customer_Labels['Churn'].value_counts()

In [ ]:
# Get proportions as percentage
Customer_Labels['Churn'].value_counts(normalize=True)

# Create Features Dataframe

Eliminate all records from 2023 to ensure there is no data leakage.

**transaction_features** will provide the record of original transactions

**Customer_Features** will aggregate the features by Customer ID and will provide the features we will need machine learning.

In [ ]:
# Create Features Dataframe
# Eliminate all records from 2023 to ensure there is no data leakage
# In this present form, it is a history of transactions, so we will name it *transaction_features*
transaction_features = superstore_cleaned_df[superstore_cleaned_df['Order Date'] < '2023-01-01']
transaction_features.sample(25)

In [ ]:
# Get features dataframe info
transaction_features.info()

In [ ]:
# Double check to ensure we include NO RECORDS with Order Date in 2023
# Get max order date of Features Dataframe
transaction_features['Order Date'].max()

In [ ]:
# TRIPLE check to ensure we include NO RECORDS with Order Date in 2023
# Sort Features Dataframe from Latest Order Date and viewing head()
transaction_features.sort_values(by='Order Date', ascending=False).head()

# Create and Explore Numeric and Date Features

We will use **transaction_features** to create **Customer_Features**.

- **transaction_features** = Dataset of transactions excluding 2023 target year

- **Customer_Features** = Dataset grouped by Customer ID, where we will collect our final features for machine learning

## Create Numeric Features
1. **Transactions:** Row ID Count
1. **Avg Quantity:** Quantity Mean
1. **Avg Sales:** Sales Mean
1. **Total Sales:** Sales Sum
1. **Avg Profit:** Profit Mean
1. **Total Profit:** Profit Sum
1. **Avg Discount:** Discount Mean
1. **Avg Ship Cost:** Shipping Cost Mean

In [ ]:
# Groupby Customer ID and use aggregations for the earliest Order Date, the most recent Order Date,
# Mean Sales, Total Sales, Mean Profit, Total Profit, and a Count of all orders by each customer
# Use the Customer ID as the new row index

Customer_Features = transaction_features.groupby('Customer ID').agg(
    Transactions=('Row ID', 'count'),
    Avg_Quantity=('Quantity', 'mean'),
    Avg_Sales=('Sales', 'mean'),
    Total_Sales=('Sales', 'sum'),
    Avg_Profit=('Profit', 'mean'),
    Total_Profit=('Profit', 'sum'),
    Avg_Discount=('Discount', 'mean'),
    Avg_Ship_Cost=('Shipping Cost', 'mean')
)

# Display updated dataframe (optional)
Customer_Features.sample(25)

## Explore Numeric Features

**Superstore_Churn_DF** = Churn Label plus features

In [ ]:
Superstore_Churn_DF = Customer_Labels.merge(Customer_Features, on='Customer ID', how='left')
Superstore_Churn_DF.head(12)

In [ ]:
Superstore_Churn_DF.info()

In [ ]:
Superstore_Churn_DF.describe()

In [ ]:
# Define function to view feature distributions and statistics, grouped by target variable
def view_distributions(data, feature, binwidth):

  # Histplot
  plt.figure(figsize=(7,3))
  ax = sns.histplot(data=data, x=feature, hue=target, binwidth=binwidth, alpha=0.6);
  plt.title(f"{feature} Distribution by Churn", fontsize=12, fontweight='bold')

  # Boxplot
  plt.figure(figsize=(7,2))
  ax = sns.boxplot(data=data, x=feature, y=target, hue=target, orient='h');
  ax.set_xlabel('')
  ax.legend_.remove()
  plt.title(f"Boxplot", fontsize=10, fontweight='bold')

  # Boxplot without Outliers
  plt.figure(figsize=(7,1))
  ax = sns.boxplot(data=data, x=feature, y=target, hue=target, orient='h', showfliers=False);
  ax.set_xlabel('')
  ax.legend_.remove()
  plt.title(f"OUTLIERS HIDDEN", fontsize=9, fontweight='bold')

  # Show plots
  plt.show()

  # Descriptive Stats
  print('\n')
  display(data.groupby(target)[feature].describe())
  print('\n')

In [ ]:
# Define variables and call view_distributions
data = Superstore_Churn_DF
target = 'Churn'

# Update this for each feature
feature = 'Transactions'

# Leave binwidth None, then adjust it to fit the data if desired
binwidth = 1

# Call the above-defined function
view_distributions(data, feature, binwidth)

**NOTE:** The upper quartiles of Non-Churners tend to have more transactions. The lower quartiles have only 1 or 2, which is the same for Churners.

In [ ]:
feature = 'Avg_Quantity'
binwidth = 1
view_distributions(data, feature, binwidth)

**NOTE**: Churners tend to purchase in lower quantities.

In [ ]:
feature = 'Avg_Sales'
binwidth = 500
view_distributions(data, feature, binwidth)

## Create Date Features
1. **Lifespan:** difference between min and max order date
1. **Frequency:** avg difference between all order dates
1. **Days to Ship:** avg days between order date and ship date
1. **Recency:** difference between max order date and target date 2023-01-01

In [ ]:
customer_date_features = transaction_features.groupby('Customer ID').agg(
    Lifespan=('Order Date', lambda x: (x.max() - x.min()).days),
    Frequency=('Order Date', lambda x: int(np.mean(np.diff(x.sort_values()).astype('timedelta64[D]').astype(int))) if len(x) > 1 else 0),
    Days_to_Ship=('Order Date', lambda x: np.mean((transaction_features.loc[x.index,'Ship Date'] - transaction_features.loc[x.index,'Order Date']).dt.days)),
    Recency=('Order Date', lambda x: (pd.to_datetime('2023-01-01') - x.max()).days)
)

# Handle potential errors (e.g., a single order date)
customer_date_features['Frequency'] = customer_date_features['Frequency'].fillna(0)
customer_date_features['Days_to_Ship'] = customer_date_features['Days_to_Ship'].fillna(0)

# Display updated dataframe (optional)
customer_date_features.sample(25)

In [ ]:
customer_date_features.info()

In [ ]:
customer_date_features.describe()

## Combine Numeric and Date Features

In [ ]:
Superstore_Churn_DF = Superstore_Churn_DF.merge(customer_date_features, on='Customer ID', how='left')
Superstore_Churn_DF.head(12)

## Explore Date Features

In [ ]:
# Continue using the above-defined function, repeated here for ease of memory
# Define function to view feature distributions and statistics, grouped by target variable
def view_distributions(data, feature, binwidth):

  # Histplot
  plt.figure(figsize=(7,3))
  ax = sns.histplot(data=data, x=feature, hue=target, binwidth=binwidth, alpha=0.6);
  plt.title(f"{feature} Distribution by Churn", fontsize=12, fontweight='bold')

  # Boxplot
  plt.figure(figsize=(7,2))
  ax = sns.boxplot(data=data, x=feature, y=target, hue=target, orient='h');
  ax.set_xlabel('')
  ax.legend_.remove()
  plt.title(f"Boxplot", fontsize=10, fontweight='bold')

  # Boxplot without Outliers
  plt.figure(figsize=(7,1))
  ax = sns.boxplot(data=data, x=feature, y=target, hue=target, orient='h', showfliers=False);
  ax.set_xlabel('')
  ax.legend_.remove()
  plt.title(f"OUTLIERS HIDDEN", fontsize=9, fontweight='bold')

  # Show plots
  plt.show()

  # Descriptive Stats
  print('\n')
  display(data.groupby(target)[feature].describe())
  print('\n')

In [ ]:
# Use this format to call the function
# Define variables and call view_distributions
data = Superstore_Churn_DF
target = 'Churn'

# Update this for each feature
feature = 'Lifespan'

# Leave binwidth None, then adjust it to fit the data if desired
binwidth = None

# Call the above-defined function
view_distributions(data, feature, binwidth)

In [ ]:
feature = ''

# Leave binwidth None, then adjust it to fit the data if desired
binwidth = None

# Call the above-defined function
view_distributions(data, feature, binwidth)

## Check for Multicollinearity

In [ ]:
# Calculate the correlation matrix
corr = Superstore_Churn_DF.corr(method='pearson')

# Create a mask to hide the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the plot
plt.figure(figsize=(12, 8))
plt.title("Correlation Heatmap", fontsize=18, fontweight='bold')

# Draw the heatmap
ax = sns.heatmap(
    corr,
    mask=mask,
    cmap=cpd,
    vmin=-1,
    vmax=1,
    annot=True,
    fmt=".2f",                  # Format annotations to 2 decimal places
    annot_kws={"size": 10},      # Optional: Adjust annotation font size
    cbar_kws={"shrink": 0.8}     # Optional: Make the colorbar a little smaller
)

# Improve layout
plt.tight_layout()
plt.show()


**NOTES:**

None are above 0.8.

These are above 0.7. We may want to remove one of these pairs in a next iteration:
  - Avg_Ship_Cost : Avg_Sales = .79
  - Total_Profit : Avg_Profit = .77
  - Frequency : Lifespan = .77

# Prepare Final Dataframe for Machine Learning


In [ ]:
# Revisit Labels Dataframe
Customer_Labels.head(12)

In [ ]:
# Merge Features Dataframe and Labels Dataframe
# Use Customer ID as the common key
Superstore_Churn_DF = Customer_Labels.merge(Customer_Features, on='Customer ID')
Superstore_Churn_DF.sample(20)

In [ ]:
# Get an overview of our combined dataframe
Superstore_Churn_DF.info()

In [ ]:
# Get numbers of churn vs non-churners
Superstore_Churn_DF['Churn'].value_counts()

In [ ]:
# Get percentages of churn vs non-churners
Superstore_Churn_DF['Churn'].value_counts(normalize=True)

# Machine Learning

We will check the quality of our feature engineering by creating predictions using an efficient and effective machine learning model:

The Random Forest Classifier from Scikit Learn

In [ ]:
# Take our prepared dataframe and create X as the features set and y as the labels
X = Superstore_Churn_DF.drop(columns=['Churn'])
y = Superstore_Churn_DF['Churn']

In [ ]:
# Preview the features we'll use for machine learning
X.head(12)

In [ ]:
# Preview the labels we'll use for machine learning
y.head(12)

In [ ]:
# Import libraries and packages for prediction and evaluation

# For creating train/test splits
from sklearn.model_selection import train_test_split

# For the Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

# For evaluating a model's predictive performance
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, confusion_matrix


In [ ]:
# Create a train / test split, with 30% of the data for test.

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# See the shape of each split compared to the total records: the number and proportion of train and test features and their labels
print(f'Total Records: {len(y)}')
print(f'Train Split: {X_train.shape[0]} Records, {len(y_train)} Labels = {round(len(y_train)/len(y), 4) * 100}%')
print(f'Test Split: {X_test.shape[0]} Records, {len(y_test)} Labels = {round(len(y_test)/len(y), 4) * 100}%')

# Preview the training data
X_train.head()

In [ ]:
# Predict churn using the Random Forest classification model from scikit learn

# Set Random Forest classifier as the algorithm for this prediction
algorithm = RandomForestClassifier(random_state=42)

# Train a prediction model by fitting the algorithm to the training set
model = algorithm.fit(X_train, y_train)

# Use the trained model to make predictions on the test set
y_pred = model.predict(X_test)

print('Predictions Complete\n')

# Dataframe of predicted churn probabilities and churn predictions (0 or 1) per customer
results_df = X_test.copy()
results_df['churn'] = y_test
results_df['predicted_probability'] = model.predict_proba(X_test)[:, 1]
results_df['prediction'] = y_pred

results_df.sample(30)

# Machine Learning Prediction Performance Metrics

In [ ]:
# Report the accuracy, precision, and recall scores of the model

accuracy = accuracy_score(y_test, y_pred) * 100
precision = precision_score(y_test, y_pred) * 100
recall = recall_score(y_test, y_pred) * 100

print('RESULTS')
print(f'Accuracy: \t{accuracy:.3f}%')
print(f'Precision: \t{precision:.3f}%')
print(f'Recall: \t{recall:.3f}%')

In [ ]:
# Generate a better custom Confusion Matrix

accuracy = accuracy_score(y_test, y_pred) * 100
precision = precision_score(y_test, y_pred) * 100
recall = recall_score(y_test, y_pred) * 100

cm = confusion_matrix(y_test, y_pred)
# Normalize true, pred, or all
cm_norm = confusion_matrix(y_test, y_pred, normalize='all')
cm_colors = sns.color_palette(['gainsboro', 'cornflowerblue'])

# axis labels for the confusion matrix plot
cm_y_labels = ['0','1'] # column labels
cm_x_labels = ['0','1'] # row labels

# Confusion matrix labels
# Review and update to match the appropriate labels for your data set
group_names = ['True Negative', 'False Positive', 'False Negative', 'True Positive']
group_counts = ['{0:0.0f}'.format(value) for value in cm.flatten()]
group_percentages = ['{0:.2%}'.format(value) for value in cm_norm.flatten()]
group_labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in
          zip(group_names, group_percentages, group_counts)]
group_labels = np.asarray(group_labels).reshape(2,2)

# Begin plot setup
fig, ax = plt.subplots(figsize=(4.2, 4.2))

# Heatmap
sns.heatmap(np.eye(2), annot=group_labels, annot_kws={'size': 11}, fmt='',
            cmap=cm_colors, cbar=False,
            yticklabels=cm_y_labels, xticklabels=cm_x_labels, ax=ax)

# Axis elements
ax.xaxis.tick_top()
ax.xaxis.set_label_position('top')
ax.tick_params(labelsize=10, length=0)
ax.set_xlabel('Predicted Values', size=10)
ax.set_ylabel('Actual Values', size=10)

# Position group labels and set colors
for text_elt, group_label in zip(ax.texts, group_labels):
    ax.text(*text_elt.get_position(), '\n', color=text_elt.get_color(),
            ha='center', va='top')

# Title for each plot
# Adjust pad to provide room for the score report below title and above confusion matrix plot
plt.title(f'{model}', pad=80, loc='left', fontsize=16, fontweight='bold')

# Score reports beneath each title
# Adjust x and y to fit report
plt.figtext(0.21, 0.81, f'{accuracy:.3f}%  Accuracy\n{precision:.3f}%  Precision\n{recall:.3f}%  Recall', wrap=True, ha='left', fontsize=10)

# Disply the plot!
plt.tight_layout()
plt.subplots_adjust(left=0.2)
print('\n') # Add a blank line for improved spacing
plt.show()

# Feature Importances

An approximate measure of relevance for each feature: The proportional influence each feature had in the prediction process.



In [ ]:
# Get feature importances from the trained RandomForestClassifier model
feature_importances = model.feature_importances_

# Create a DataFrame to store feature names and their importances
feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})

# Sort the DataFrame by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Make Feature the new index
feature_importance_df = feature_importance_df.set_index('Feature')

# Print the features and their importances
print("Feature Importances:")
feature_importance_df
